In [1]:
import openai
import time


try:
    from local_settings import OPENAI_API_KEY_GPT4
except:
    ### If you dont want to setup a local_settings file, you can add your key here
    OPENAI_API_KEY_GPT4 = ""

openai.api_key = OPENAI_API_KEY_GPT4

In [2]:
def _get_gpt_response(prompt):
    try:
        response = openai.ChatCompletion.create(
            model='gpt-4', 
            messages=prompt, 
            temperature=0.9, 
            max_tokens=1000,
            frequency_penalty=1,
            presence_penalty=1
        )
    except Exception as e:
        print(f'OpenAI API error, sleeping and retrying... {e}')
        time.sleep(3)
        response = openai.ChatCompletion.create(
            model='gpt-4', 
            messages=prompt, 
            temperature=0.9,  
            max_tokens=1000,
            frequency_penalty=1,
            presence_penalty=1
        )
    text = response.choices[0]['message']['content'].strip()
    output = {
        'usage': response['usage'],
        'text': text
    }

    output['usage']['usd'] = (output['usage']['total_tokens'] / 1000) * 0.03
    return output


In [3]:
def write_letter():
    with open('job_description.txt', 'r') as f:
        job_description = f.read()
    
    with open('cover_letter.txt', 'r') as f:
        example_cover_letter = f.read()

    with open('resume.txt', 'r') as f:
        resume = f.read()

    prompt = [
{
    'role': 'system',
    'content': f"""
You are CoverLetterGPT. You are trained to help users write effective, concise cover letters for jobs based on their experiences and the job description. 

You write concise, 3 paragraph cover letters communicating user's skills. Even if the provided example cover letter is longer than 3 paragraphs, you only write 3 concise paragraphs.
    
The 3 paragraphs in your cover letters have the following content:
- introduction
    - Broadly describe the user's experience and why they are a good fit for the job. Mention relevant previous work without going into too much detail
- experiences
    - Specifically describe the user's experiences and how they relate to the job description. Mention relevant previous work in detail
- conclusion
    - Thank the hiring manager for their time and express interest in the job. Keep this simple and concise.

You do not write more than 3 concise paragraphs. 

You copy the tone of the author of the example cover letter provided to you. Try to use a similar level of formality and professionalism as the example cover letter. 

You do not copy the length of the example cover letter. You only write 3 concise paragraphs.
    
Relate the user's experience to the job description and make the user sounds like a good fit for the job without making up experiences they had or requirements not mentioned in the job description. 

Make sure to not overstate the user's experience and refer to the resume and job description for details.
    
Output the entire cover letter to the user so they can copy and paste it. Be concise, engaging and professional. 

Do not write more than 3 concise paragraphs.

You are currently writing cover letters for a user named Matt. Matt is a 26 year old machine learning engineer with degrees in biology and bioinformatics. Matt has worked as a machine learning engineer at a web3 company, a data scientist at a mental health tech company and a data engineer at a valued based healthcare company. 

Matt is hard-working, curious, passionate about machine learning and making the world better. Matt also avoids flowery words like "thrilled", "profound", "spearheaded", "intricate" and prefers simpler, more direct phrasing. 
"""
    },
    {
        'role': 'user',
        'content': f"Here's my resume"
    },
    {
        'role': 'user',
        'content': f'```{resume}```'
    },
    {
        'role': 'user',
        'content': f"Here's an example cover letter I wrote for another job I applied for"
    },
    {
        'role': 'user',
        'content': f'```{example_cover_letter}```'
    },
    {
        'role': 'user',
        'content': f"Please write me a 3 paragraph cover letter for this job:"
    },
    {
        'role': 'user',
        'content': f'```{job_description}```'
    },
    ]
    print(prompt)
    response = _get_gpt_response(prompt)
    print(response['usage'])
    parsed_output = response["text"]
    return parsed_output

def update_letter(new_cover_letter):
    with open('job_description.txt', 'r') as f:
        job_description = f.read()
    
    with open('cover_letter.txt', 'r') as f:
        example_cover_letter = f.read()
    
    with open('resume.txt', 'r') as f:
        resume = f.read()

    prompt = [
{
    'role': 'system',
    'content': f"""
You are CoverLetterGPT. You are trained to help users write cover letters for jobs based on their experiences and the job description. You wrote a 3 paragraph cover letter already and now you're editing it. Your main task is to make it concise and easy to read. 

Below are the requirements you were given originally:
    
The 3 paragraphs in your cover letters have the following content:
- introduction
    - Broadly describe the user's experience and why they are a good fit for the job
- experiences
    - Specifically describe the user's experiences and how they relate to the job description
- conclusion
    - Thank the hiring manager for their time and express interest in the job

You do not write more than 3 concise paragraphs. 

You copy the tone of the author of the example cover letter provided to you. Try to use a similar level of formality and professionalism as the example cover letter. You do not copy the length of the example cover letter. You only write 3 concise paragraphs.
    
Relate the user's experience to the job description and make the user sounds like a good fit for the job without making up experiences they had. Make sure to not overstate the user's experience and refer to the resume for details.
    
Output the entire cover letter to the user so they can copy and paste it. Be concise, engaging and professional. 

Do not write more than 3 paragraphs.

You are currently writing cover letters for a user named Matt. Matt is a 26 year old machine learning engineer with degrees in biology and bioinformatics. Matt has worked as a machine learning engineer at a web3 company, a data scientist at a mental health tech company and a data engineer at a valued based healthcare company. 

Matt is hard-working, curious, passionate about machine learning and making the world better. Matt also avoids flowery words like "thrilled", "profound", "spearheaded", "intricate" and prefers simpler, more direct phrasing. 
"""
    },
    {
        'role': 'user',
        'content': f"Here's my resume"
    },
    {
        'role': 'user',
        'content': f'```{resume}```'
    },
    {
        'role': 'user',
        'content': f"Here's an example cover letter I wrote for another job I applied for"
    },
    {
        'role': 'user',
        'content': f'```{example_cover_letter}```'
    },
    {
        'role': 'user',
        'content': f"Here's the job description for this role"
    },
    {
        'role': 'user',
        'content': f'```{job_description}```'
    },
    {
        'role': 'assistant',
        'content': f"""Here's my first version: 
```{new_cover_letter}```"""
    },
    {
        'role': 'user',
        'content': f"Great now can you make it more concise and easier to read?"
    },
    ]
    print(prompt)
    response = _get_gpt_response(prompt)
    print(response['usage'])
    parsed_output = response["text"]
    return parsed_output

output = write_letter(
    example_cover_letter=cover_letter,
)

output = update_letter(
    example_cover_letter=cover_letter,
    new_cover_letter=output,
)

with open('output.txt', 'w') as f:
    f.write(output)

NameError: name 'cover_letter' is not defined